In [1]:
import os
import cv2
import json
import nltk
import spacy
import imutils
from nltk import *
import numpy as np
import moviepy.editor
import es_core_news_sm
from collections import Counter
from nltk.corpus import wordnet
from spacy.matcher import matcher
from spacy.lang.es import Spanish
import speech_recognition as sr
from moviepy.editor import VideoFileClip
from keras import backend as K
from keras.models import load_model
from keras.models import model_from_json
from keras.preprocessing.image import img_to_array
from spacy_wordnet.wordnet_annotator import WordnetAnnotator
from nltk.sentiment.vader import SentimentIntensityAnalyzer

Using TensorFlow backend.


In [2]:
def videoAnalyze(video):
    '''
    Video-analisis con reconocimiento sobre el género.
    '''
    with open('../output/model_sequential84.16370153427124.json', 'r') as f:
        model_json = json.load(f)

    model = model_from_json(model_json)
    model.load_weights(
        '../output/model_sequential84.16370153427124.h5')

    face_cascade = cv2.CascadeClassifier(
        '../haarcascade_frontalface_default.xml')

    cap = cv2.VideoCapture(video)
    result = []

    while cap.isOpened():
        frame = cap.read()[1]

        try:
            frame = imutils.resize(frame, width=400)
            color = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            face = face_cascade.detectMultiScale(
                color,
                scaleFactor=1.1,
                minNeighbors=5
            )

            frameClone = frame.copy()

            if len(face) > 0:
                (x, y, w, h) = face[0]
                roi = color[y:y+h, x:x+w]
                roi = cv2.resize(roi, (60, 60))
                roi = np.stack(roi)

                p = np.expand_dims(roi, axis=0).reshape(
                    np.expand_dims(roi, axis=0).shape[0], 60, 60, 1)

                genre = model.predict(p)[0]
                label = ['man', 'woman'][np.argmax(genre)]

                result.append(label)

                if genre.max() > 0.80:
                    cv2.rectangle(frameClone, (x, y),
                                  (x + w, y + h), (0, 255, 0), 1)
                    cv2.putText(frameClone, label, (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)

            cv2.imshow('Advertising Analyze',
                       imutils.resize(frameClone, width=450))

        except:
            cap.release()

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    m = result.count('man')
    w = result.count('woman')

    return print('man -> ', "{0:.2f}%".format((m/(m+w))*100), '// woman -> ', "{0:.2f}%".format((w/(m+w))*100))

In [3]:
def extractAudio(video):
    '''
    Transforma el archivo video a un archivo audio, devuelve el audio en formato texto.
    '''
    vFile = moviepy.editor.VideoFileClip(video)
    aFile = vFile.audio

    save = f"{video[:-4]}.wav"
    aFile.write_audiofile(save)

    re = sr.Recognizer()

    with sr.AudioFile(save) as source:
        info_audio = re.record(source)
        text = re.recognize_google(info_audio, language='es-ES')

    os.remove(save)
    print('\nIMPRIMIENDO AUDIO...')
        
    return text

In [4]:
def extractTime(video):
    '''
    Extraer la duración de un video.
    '''
    clip = VideoFileClip(video)
    
    return int(clip.duration)

In [5]:
def textSentiment(text):
    '''
    Realiza un análisis de sentimiento sobre un texto.
    '''
    s = SentimentIntensityAnalyzer()
    result = s.polarity_scores(text)
    
    return result

In [6]:
def textAnalysis(text):
    '''
    Realiza un nálisis textual.
    '''
    nlp = es_core_news_sm.load()
    
    complete_doc = nlp(text)
    words = [token.text for token in complete_doc if not token.is_stop and not token.is_punct]
    
    word_freq = Counter(words)
    common_words = word_freq.most_common(5)
    
    nouns = []
    adjectives = []
    
    for token in complete_doc:
        print(token,'-', token.tag_, token.pos_)
    
    for token in complete_doc:
        if token.pos_ == 'NOUN':
            nouns.append(token)
        if token.pos_ == 'ADJ':
            adjectives.append(token)
            
    print('\nPALABRAS REPETIDAS...')
    print(common_words)
    print('\nNOMBRES...')
    print(nouns)
    print('\nADJETIVOS...')
    print(adjectives)
    
    

In [7]:
video = '../input/video_prueba1.mpg'

In [8]:
print('VISUALIZANDO SPOT...')
print('PORCENTAJES DE REGISTROS POR GÉNERO...')
print(videoAnalyze(video), '\n')
print('EXTRAYENDO DURACIÓN DEL SPOT...')
print(extractTime(video), 'seg', '\n')
print('EXTRAYENDO AUDIO...')
ea = [extractAudio(video)]
print(ea, '\n')
print('ANÁLISIS SENTIMENTAL...')
print(textSentiment(ea[0]), '\n')
print('ANÁLISIS TEXTUAL...')
print(textAnalysis(ea[0]))

VISUALIZANDO SPOT...
PORCENTAJES DE REGISTROS POR GÉNERO...
man ->  26.25% // woman ->  73.75%
None 

EXTRAYENDO DURACIÓN DEL SPOT...
10 seg 

EXTRAYENDO AUDIO...


chunk:  46%|████▌     | 102/221 [00:00<00:00, 955.44it/s, now=None]

MoviePy - Writing audio in ../input/video_prueba1.wav


MoviePy - Done.

IMPRIMIENDO AUDIO...
['por si necesitan algo auténtico dáselo con nuestra marca y mercado de Aldi frescos muy frescos y por mucho menos que la vida te pille preparado'] 

ANÁLISIS SENTIMENTAL...
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} 

ANÁLISIS TEXTUAL...
por - ADP__AdpType=Prep ADP
si - SCONJ___ SCONJ
necesitan - VERB__Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin VERB
algo - PRON__Number=Sing|PronType=Ind PRON
auténtico - ADJ__Gender=Masc|Number=Sing ADJ
dáselo - NOUN__Gender=Masc|Number=Sing NOUN
con - ADP__AdpType=Prep ADP
nuestra - DET__Gender=Fem|Number=Sing|Number[psor]=Plur|Person=1|Poss=Yes|PronType=Prs DET
marca - NOUN__Gender=Fem|Number=Sing NOUN
y - CCONJ___ CONJ
mercado - NOUN__Gender=Masc|Number=Sing NOUN
de - ADP__AdpType=Prep ADP
Aldi - PROPN___ PROPN
frescos - ADJ__Gender=Masc|Number=Plur ADJ
muy - ADV___ ADV
frescos - ADJ__Gender=Masc|Number=Plur ADJ
y - CCONJ___ CONJ
por - ADP__AdpType=Prep ADP
mucho - ADV___ ADV
menos - ADV___